In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

Solar interior
==========

We start by considering properties of the solar interior, based on Model S.  We clip the solar interior at $r/R_\odot = 0.99$, to avoid possible boundary layer issues.

In [ ]:
col_names = ['r', 'c', 'rho', 'p', 'Gamma_1', 'T']
data = pd.read_csv('model_S.txt', delim_whitespace=True, 
                   comment='#', names=col_names)
mask = (data['r']<=0.99)
data = data[mask]

In [ ]:
fig, ax = plt.subplots()
ax.plot(data['r'], np.log(data['rho']))
ax.set_ylabel(r'$\ln \rho$')
ax.set_xlabel(r'$r/R_\odot$')
ax2 = ax.twinx()
γ = data['Gamma_1']
s = 1/γ*np.log(data['T'])-(γ-1)/γ*np.log(data['rho'])
ax2.plot(data['r'], s, color='tab:orange')
ax2.set_ylabel(r'$s/c_P$')
ax2.yaxis.label.set_color('tab:orange')
ax2.tick_params(axis='y', colors='tab:orange')

In [ ]:
k_B = 1.3e-16 # ergs/K
q_e = 4.8e-10 # statcoul or esu
N_A = 6.0221e23 # Avagadro's number
m_H = 1/(N_A) # 1g/mol -> mass per atom via N_A
n = data['rho']/m_H # approximation to 100% Hydrogen
T = data['T']
λ_D = np.sqrt(k_B*T/(8*np.pi*n*q_e**2))
fig, ax = plt.subplots()
ax.plot(data['r'], λ_D)
ax.set_yscale('log')
ax.set_ylabel(r'$\lambda_D$ [cm]')
ax.set_xlabel(r'$r/R_\odot$')

Next we compute Braginskii plasma diffusivities, following the NRL plasma formulary.  We use $\eta^i_0$ as the proxy for unmagentized plasmas, following the $P_{jk} = -\eta_0 W_{jk}$ notation, where \begin{equation}
W = \nabla u + (\nabla u)^T - 2/3 \mathrm{Tr}(\nabla\cdot u)
\end{equation}
which 
\begin{equation}
    \eta^i_0 = 0.96 n k T \tau_i
\end{equation}
with
\begin{equation}
    \tau_i = \frac{3\sqrt{m_i} (k T)^{3/2}}{4 \sqrt{\pi} n \lambda e^4}
\end{equation}
and where $\lambda$ here is the coulomb logarithm, where typically $\lambda \approx 10-20$.

The units on $\eta_0$ are in $g/cm/s$, so $\eta_0 = \mu$ and $\nu = \eta_0/\rho$.  This aligns with the NRL expression of the momentum equation (pg 36).

In [ ]:
λ = 10
τ_i = (3*np.sqrt(m_H)*(k_B*T)**(3/2))/(4*np.sqrt(np.pi)*n*λ*q_e**4)
η_0 = 0.96*n*k_B*T*τ_i
ν = η_0/data['rho']

In [ ]:
fig, ax = plt.subplots()
ax.plot(data['r'], ν)
ax.set_yscale('log')
ax.set_ylabel(r'$\nu$ [cm$^2$/s]')
ax.set_xlabel(r'$r/R_\odot$')

The magnetic diffusion coefficient (also confusingly $\eta$) is given by:
\begin{equation}
    \eta = \frac{c^2}{4 \pi \sigma} ~\mathrm{cm}^2\:\mathrm{s}^{-1}
\end{equation}
with
\begin{equation}
  \sigma \approx \sigma_\parallel = 1.96 \sigma_\perp 
  = 1.96 \frac{Z n_i e^2 \tau_e}{m_e}~\mathrm{s}^{-1}
\end{equation}
and
\begin{equation}
  \eta = \frac{c^2}{4 \pi} \frac{m_e}{1.96 Z n_i e^2 \tau_e}~\mathrm{cm}^2\:\mathrm{s}^{-1}.
\end{equation}

In [ ]:
c = 3e10
m_e = 9.11e-28
τ_e = (3*np.sqrt(m_e)*(k_B*T)**(3/2))/(4*np.sqrt(np.pi)*n*λ*q_e**4)

η = c**2/(4*np.pi)*m_e/(1.96*n*q_e**2*τ_e)
fig, ax = plt.subplots()
ax.plot(data['r'], ν/η)
ax.set_yscale('log')
ax.set_ylabel(r'$\mathrm{Pm}=\nu/\eta$')
ax.set_xlabel(r'$r/R_\odot$')

Solar atmosphere
==============
We next consider properties of the solar atmosphere, based on the VAL atmosphere model.  Here, we narrowly consider the lower atmosphere, and in particular the region up to about 3000km (3Mm).

In [ ]:
col_names =['cell', 'z','column_density',  'T', 'V' , 'p_g', 'p_tot','n_H','n_HI','n_e']
data = pd.read_csv('solar_atm.txt', delim_whitespace=True, 
                   comment='#', names=col_names)
mask = (data['z']<3000)
data = data[mask]

n = data['n_H']+data['n_HI']
ρ = m_H*n

In [ ]:
fig, ax = plt.subplots()
ax.plot(data['z'], np.log(ρ))
ax.set_ylabel(r'$\ln \rho$')
ax.set_xlabel(r'$z$ [km]')
ax2 = ax.twinx()
γ = 5/3
s = 1/γ*np.log(data['T'])-(γ-1)/γ*np.log(ρ)
ax2.plot(data['z'], s, color='tab:orange')
ax2.set_ylabel(r'$s/c_P$')
ax2.yaxis.label.set_color('tab:orange')
ax2.tick_params(axis='y', colors='tab:orange')

In [ ]:
T = data['T']
n_e = data['n_e']
λ_D = np.sqrt(k_B*T/(8*np.pi*n*q_e**2))
λ_D_e = np.sqrt(k_B*T/(8*np.pi*n_e*q_e**2))
fig, ax = plt.subplots()
ax.plot(data['z'], λ_D, label=r'$\lambda_{D}$')
ax.plot(data['z'], λ_D_e, label=r'$\lambda_{D,e}$')
ax.set_yscale('log')
ax.set_ylabel(r'$\lambda_D$ [cm]')
ax.set_xlabel(r'$z$ [km]')
ax.legend()

In [ ]:
λ = 10
τ_i = (3*np.sqrt(m_H)*(k_B*T)**(3/2))/(4*np.sqrt(np.pi)*n*λ*q_e**4)
η_0 = 0.96*n*k_B*T*τ_i
ν = η_0/ρ

fig, ax = plt.subplots()
ax.plot(data['z'], ν)
ax.set_yscale('log')
ax.set_ylabel(r'$\nu$ [cm$^2$/s]')
ax.set_xlabel(r'$z$ [km]')

In [ ]:
c = 3e10
m_e = 9.11e-28
τ_e = (3*np.sqrt(m_e)*(k_B*T)**(3/2))/(4*np.sqrt(np.pi)*n_e*λ*q_e**4)

η = c**2/(4*np.pi)*m_e/(1.96*n*q_e**2*τ_e)
fig, ax = plt.subplots()
ax.plot(data['z'], ν/η)
ax.set_yscale('log')
ax.set_ylabel(r'$\mathrm{Pm}=\nu/\eta$')
ax.set_xlabel(r'$z$ [km]')